# 1. Installing and importing required packages

**Librosa** is a python package for music and audio analysis. 
It provides the building blocks necessary to create music information retrieval systems.

In [ ]:
! pip install librosa

In [ ]:
! pip install -q efficientnet

In [ ]:
# Default Libraries required
import numpy as np
import pandas as pd

import math, os, re, warnings, random
import tensorflow as tf

import librosa
import librosa.display

from kaggle_datasets import KaggleDatasets

#Visualization Libraries
import matplotlib.pyplot as plt
import seaborn as sns

#File Accessing Libraries
import os
import os.path
from os import path

import time

from IPython.display import Audio
import IPython.display as ipd

from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold

from tensorflow.keras import Model, layers
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler
from tensorflow.keras.layers import GlobalAveragePooling2D, Input, Dense, Dropout, GaussianNoise
from tensorflow.keras.applications import ResNet50
import tensorflow.keras.backend as K

import efficientnet.keras as efn

In [ ]:
print(tf.__version__)

In [ ]:
def seed_everything(seed=0):
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    os.environ['TF_DETERMINISTIC_OPS'] = '1'

seed = 42
seed_everything(seed)
warnings.filterwarnings('ignore')  

In [ ]:
# Detect hardware, return appropriate distribution strategy
try:
    # TPU detection. No parameters necessary if TPU_NAME environment variable is
    # set: this is always the case on Kaggle.
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
    strategy = tf.distribute.get_strategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

In [ ]:
AUTO = tf.data.experimental.AUTOTUNE
REPLICAS = strategy.num_replicas_in_sync

In [ ]:
# For tf.dataset
AUTO = tf.data.experimental.AUTOTUNE

# Configuration
EPOCHS = 10
BATCH_SIZE = 8 * strategy.num_replicas_in_sync

# 2. Data Exploration

In [ ]:
traindf=pd.read_csv(r"/kaggle/input/rfcx-species-audio-detection/train_tp.csv")
traindf.shape

In [ ]:
traindf.head()

In [ ]:
print(f"total Species in Dataset : {len(traindf.recording_id)}")
print(f"total Recordings in Dataset : {len(traindf.recording_id.unique())}")

In [ ]:
sns.countplot(x=traindf['species_id'])

Species_id is balanced, all the categories have sufficient number of entries.<br/>
Let's look into other features

In [ ]:
sns.countplot(x=traindf['songtype_id'])

In [ ]:
path='/kaggle/input/rfcx-species-audio-detection/train/003bec244.flac'
y, sr = librosa.load(path)
plt.figure(figsize=(20,5))
librosa.display.waveplot(y, sr=sr)

In [ ]:
#display Spectrogram
path='/kaggle/input/rfcx-species-audio-detection/train/003bec244.flac'
x, sr = librosa.load(path)
X = librosa.stft(x)
Xdb = librosa.amplitude_to_db(abs(X))
plt.figure(figsize=(20, 5))
librosa.display.specshow(Xdb, sr=sr, x_axis='time', y_axis='hz') 
#If to pring log of frequencies  
#librosa.display.specshow(Xdb, sr=sr, x_axis='time', y_axis='log')
plt.colorbar()

* Using TFRecord Data for model building to leverage the features of Tensorflow.<br/>
* The TFRecord format is a simple format for storing a sequence of binary records.<br/>
* To Explore more about TFRecord, associated methods and functions, please refer -https://www.tensorflow.org/tutorials/load_data/tfrecord


In [ ]:
def _bytes_feature(value):
  """Returns a bytes_list from a string / byte."""
  if isinstance(value, type(tf.constant(0))):
    value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
  """Returns a float_list from a float / double."""
  return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def serialize_data(wav, recording_id, target, song_id, tmin,fmin, tmax, fmax):
  feature = {
      'wav': _bytes_feature(wav),
      'recording_id': _bytes_feature(recording_id),
      'target': _float_feature(target),
      'song_id': _float_feature(song_id),
      'tmin': _float_feature(tmin),
      'fmin' : _float_feature(fmin),
      'tmax': _float_feature(tmax),
      'fmax' : _float_feature(fmax),
  }
  proto_buff = tf.train.Example(features=tf.train.Features(feature=feature))
  return proto_buff.SerializeToString() 

from sklearn.model_selection import StratifiedKFold

tfrec_num = 0
kfold = StratifiedKFold(n_splits=10, shuffle=False)
for fold, (train_idx, test_idx) in enumerate(kfold.split(traindf['recording_id'], traindf['species_id'])):
    x_train , y_train = traindf['recording_id'][test_idx] , traindf['species_id'][test_idx]
   
    with tf.io.TFRecordWriter('tp%.2i-%.2i.tfrec'%(tfrec_num, len(test_idx))) as writer:
        print('Writing_tfrecords ',fold)
        for recording_id , true_value in zip(x_train, y_train): 
            wav, _ = librosa.load(f'../input/rfcx-species-audio-detection/train/{recording_id}.flac', sr = None)
            label_info = traindf.loc[traindf['recording_id'] == str(recording_id)].values[0]
            wav = tf.audio.encode_wav(tf.reshape(wav,(wav.shape[0], 1)) ,sample_rate = 48000)
            recording_id = label_info[0].encode()
            target = label_info[1]
            song_id = label_info[2]
            tmin = label_info[3]
            fmin = label_info[4]
            tmax = label_info[5]
            fmax = label_info[6]
            proto_buffer = serialize_data(wav, recording_id, target, song_id, tmin,fmin, tmax, fmax)
            writer.write(proto_buffer)
    tfrec_num += 1

The above step will generate TFRecords in Kaggle/Working folder, however these files can't be used in TPU processing as TPU works only on Public dataset which are part of Input folder. so these newly created TFrecords are to be stored in Public datasets and have to be imported.

In [ ]:
AUTO = tf.data.experimental.AUTOTUNE
ignore_order = tf.data.Options()
ignore_order.experimental_deterministic = False
    
# On Kaggle you can also use KaggleDatasets().get_gcs_path() to obtain the GCS path of a Kaggle dataset

TRAIN_DATA_DIR = 'rfcx-audio-detection'
TRAIN_GCS_PATH = KaggleDatasets().get_gcs_path(TRAIN_DATA_DIR)
FILENAMES = tf.io.gfile.glob(TRAIN_GCS_PATH + '/tp*.tfrec')

dataset = tf.data.TFRecordDataset(FILENAMES, num_parallel_reads=AUTO)
dataset = dataset.with_options(ignore_order)

In [ ]:
#test_files
TEST_DATA_DIR = 'rfcx-species-audio-detection'
TEST_GCS_PATH =  KaggleDatasets().get_gcs_path(TEST_DATA_DIR)
TEST_FILES = tf.io.gfile.glob(TEST_GCS_PATH + '/tfrecords/test/*.tfrec')

In [ ]:
n = [int(re.compile(r"-([0-9]*)\.").search(filename).group(1)) for filename in FILENAMES]
print(f"Total number of files :{np.sum(n)}")
no_of_training_samples=np.sum(n)

Configuration Variables

In [ ]:
#Model Related Configurations
CUT = 10
TIME = 10
GLOBAL_BATCH_SIZE = 16 * REPLICAS
LEARNING_RATE = 0.0015
WARMUP_LEARNING_RATE = 1e-5
WARMUP_EPOCHS = int(EPOCHS*0.1)
PATIENCE = 8
STEPS_PER_EPOCH = 128
N_FOLDS = 5
NUM_TRAINING_SAMPLES = no_of_training_samples

#Audio related Configurations
sample_rate = 48000
stft_window_seconds: float = 0.025
stft_hop_seconds: float = 0.005
frame_length: int =  1200    
mel_bands: int = 512
mel_min_hz: float = 50.0
mel_max_hz: float = 24000.0
log_offset: float = 0.001
patch_window_seconds: float = 0.96
patch_hop_seconds: float = 0.48

patch_frames =  int(round(patch_window_seconds / stft_hop_seconds))
patch_bands = mel_bands
height = mel_bands
width = 2000
num_classes: int = 24
dropout = 0.35
classifier_activation: str = 'sigmoid'

In [ ]:
feature_description_label = {
    'wav': tf.io.FixedLenFeature([], tf.string),
    'recording_id': tf.io.FixedLenFeature([], tf.string ),
    'target' : tf.io.FixedLenFeature([], tf.float32),
    'song_id': tf.io.FixedLenFeature([], tf.float32),
     'tmin' : tf.io.FixedLenFeature([], tf.float32),
     'fmin' : tf.io.FixedLenFeature([], tf.float32),
     'tmax' : tf.io.FixedLenFeature([], tf.float32),
     'fmax' : tf.io.FixedLenFeature([], tf.float32),
}
feature_description_unlabel = {
    'recording_id': tf.io.FixedLenFeature([], tf.string),
    'audio_wav': tf.io.FixedLenFeature([], tf.string),
    }
feature_dtype = {
    'wav': tf.float32,
    'recording_id': tf.string,
    'target': tf.float32,
    'song_id': tf.float32,
    't_min': tf.float32,
    'f_min': tf.float32,
    't_max': tf.float32,
    'f_max':tf.float32,
}

Cut the audio frame to required time regions

In [ ]:
def ReadLabeledAudio(protoBuff):
    sample = tf.io.parse_single_example(protoBuff, feature_description_label)
    wav, _ = tf.audio.decode_wav(sample['wav'], desired_channels=1) # mono
    target = tf.cast(sample['target'],tf.float32)
    target = tf.squeeze(tf.one_hot([target,], depth = num_classes), axis = 0)
    #get tmin,tmax fmin,fmax
    tmin = tf.cast(sample['tmin'], tf.float32)
    fmin = tf.cast(sample['fmin'], tf.float32)
    tmax = tf.cast(sample['tmax'], tf.float32)
    fmax = tf.cast(sample['fmax'], tf.float32)
    
    tmax_s = tmax * tf.cast(sample_rate, tf.float32)
    tmin_s = tmin * tf.cast(sample_rate, tf.float32)
    cut_s = tf.cast(CUT * sample_rate, tf.float32)
    all_s = tf.cast(60 * sample_rate, tf.float32)
    tsize_s = tmax_s - tmin_s
    cut_min = tf.cast(tf.maximum(0.0,tf.minimum(tmin_s - (cut_s - tsize_s) / 2,tf.minimum(tmax_s + (cut_s - tsize_s) / 2, all_s) - cut_s)), tf.int32)
    cut_max = cut_min + CUT * sample_rate
    wav = tf.squeeze(wav[cut_min : cut_max])
    
    return wav, target

In [ ]:
def ReadUnlabeledAudio(protoBuff):

    sample = tf.io.parse_single_example(protoBuff, feature_description_unlabel)
    wav, _ = tf.audio.decode_wav(sample['audio_wav'], desired_channels=1) # mono
    recording_id = tf.reshape(tf.cast(sample['recording_id'] , tf.string), [1])

    def _cut_audio(i):
        _sample = {
            'audio_wav': tf.reshape(wav[i*sample_rate*TIME:(i+1)*sample_rate*TIME], [sample_rate*TIME]),
            'recording_id': sample['recording_id']
        }
        return _sample

    return tf.map_fn(_cut_audio, tf.range(60//TIME), dtype={
        'audio_wav': tf.float32,
        'recording_id': tf.string
    })

A spectrogram shows how the frequency content of a signal changes over time and can be calculated from the time domain signal.

To know more about method - tf.signal.stft() - https://www.tensorflow.org/api_docs/python/tf/signal/stft

To know more about linear_to_mel_weight_matrix() - https://www.tensorflow.org/api_docs/python/tf/signal/linear_to_mel_weight_matrix

Transform raw audio data to Log-Mel-Spectogram
1. **Compute the short-time Fourier transform of audio signals** - STFT divides a long signal into shorter segments, often called frames, and computes the spectrum for each frame. The frames typically overlap to minimize data-loss at the edges
2. **Compute the magnitudes**- STFT from the previous step returns a tensor of complex values. Use tf.abs() to compute the magnitudes.
3. **Instantiate the mel filterbank**- Transforming standard spectrograms to mel-spectrograms involves warping frequencies to the mel-scale and combining FFT bins to mel-frequency bins.
4. **Warp the linear-scale magnitude** -spectrograms to mel-scale -Multiply the squared magnitude-spectrograms with the mel-filterbank and you get mel-scaled power-spectrograms.
5. **Transform magnitudes to log-scale** - We perceive changes in loudness logarithmically. So, in this last step, we want to scale the mel-spectrograms’ magnitudes logarithmically, too.

https://towardsdatascience.com/how-to-easily-process-audio-on-your-gpu-with-tensorflow-2d9d91360f06

In [ ]:
def power_to_db(S, amin=1e-16, top_db=80.0):
    """Convert a power-spectrogram (magnitude squared) to decibel (dB) units.
    Computes the scaling ``10 * log10(S / max(S))`` in a numerically
    stable way.
    Based on:
    https://librosa.github.io/librosa/generated/librosa.core.power_to_db.html
    """
    def _tf_log10(x):
        numerator = tf.math.log(x)
        denominator = tf.math.log(tf.constant(10, dtype=numerator.dtype))
        return numerator / denominator
    
    # Scale magnitude relative to maximum value in S. Zeros in the output 
    # correspond to positions where S == ref.
    ref = tf.reduce_max(S)

    log_spec = 10.0 * _tf_log10(tf.maximum(amin, S))
    log_spec -= 10.0 * _tf_log10(tf.maximum(amin, ref))

    log_spec = tf.maximum(log_spec, tf.reduce_max(log_spec) - top_db)

    return log_spec

In [ ]:
def transform_log_mel_spect(waveform,target):
    #1. Compute the short-time Fourier transform of your audio signals  
    window_length_samples = int(
      round(sample_rate * stft_window_seconds))
    hop_length_samples = int(
      round(sample_rate * stft_hop_seconds))
    fft_length = 2 ** int(np.ceil(np.log(window_length_samples) / np.log(2.0)))
    num_spectrogram_bins = fft_length // 2 + 1
    #2. Compute the magnitudes
    magnitude_spectrogram = tf.abs(tf.signal.stft(
      signals=waveform,
      frame_length=frame_length,
      frame_step=hop_length_samples,
      fft_length= fft_length))
    # magnitude_spectrogram has shape [<# STFT frames>, num_spectrogram_bins]

    #3.Instantiate the mel filterbank
    mel_filterbank = tf.signal.linear_to_mel_weight_matrix(
        num_mel_bins=mel_bands,
        num_spectrogram_bins=num_spectrogram_bins,
        sample_rate=sample_rate,
        lower_edge_hertz=mel_min_hz,
        upper_edge_hertz=mel_max_hz)
    mel_spectrogram = tf.matmul(
      magnitude_spectrogram, mel_filterbank)
 
    log_mel = tf.math.log(mel_spectrogram + log_offset)
#     log_magnitude_mel_spect has shape [<# STFT frames>, mel_bands]
    log_mel = tf.transpose(log_mel)
    log_magnitude_mel_spect = tf.reshape(log_mel , [tf.shape(log_mel)[0] ,tf.shape(log_mel)[1],1])
    # Frame spectrogram (shape [<# STFT frames>, mel_bands]) into patches
    # (the input examples). Only complete frames are emitted, so if there is
    # less than patch_window_seconds of waveform then nothing is emitted
    # (to avoid this, zero-pad before processing).
    spectrogram_hop_length_samples = int(
      round(sample_rate * stft_hop_seconds))
    spectrogram_sample_rate = sample_rate / spectrogram_hop_length_samples
    patch_window_length_samples = int(
      round(spectrogram_sample_rate * patch_window_seconds))
    patch_hop_length_samples = int(
      round(spectrogram_sample_rate * patch_hop_seconds))
    features = tf.signal.frame(
        signal=log_magnitude_mel_spect,
        frame_length=patch_window_length_samples,
        frame_step=patch_hop_length_samples,
        axis=0)
    # features has shape [<# patches>, <# STFT frames in an patch>, mel_bands]
    
    return log_magnitude_mel_spect, target

In [ ]:
def frequency_masking(mel_spectrogram):
    
    frequency_masking_para = 80, 
    frequency_mask_num = 2
    
    fbank_size = tf.shape(mel_spectrogram)
#     print(fbank_size)
    n, v = fbank_size[0], fbank_size[1]

    for i in range(frequency_mask_num):
        f = tf.random.uniform([], minval=0, maxval= tf.squeeze(frequency_masking_para), dtype=tf.int32)
        v = tf.cast(v, dtype=tf.int32)
        f0 = tf.random.uniform([], minval=0, maxval= tf.squeeze(v-f), dtype=tf.int32)

        # warped_mel_spectrogram[f0:f0 + f, :] = 0
        mask = tf.concat((tf.ones(shape=(n, v - f0 - f,1)),
                          tf.zeros(shape=(n, f,1)),
                          tf.ones(shape=(n, f0,1)),
                          ),1)
        mel_spectrogram = mel_spectrogram * mask
    return tf.cast(mel_spectrogram, dtype=tf.float32)


def time_masking(mel_spectrogram):
    time_masking_para = 40, 
    time_mask_num = 1
    
    fbank_size = tf.shape(mel_spectrogram)
    n, v = fbank_size[0], fbank_size[1]

   
    for i in range(time_mask_num):
        t = tf.random.uniform([], minval=0, maxval=tf.squeeze(time_masking_para), dtype=tf.int32)
        t0 = tf.random.uniform([], minval=0, maxval= n-t, dtype=tf.int32)

        # mel_spectrogram[:, t0:t0 + t] = 0
        mask = tf.concat((tf.ones(shape=(n-t0-t, v,1)),
                          tf.zeros(shape=(t, v,1)),
                          tf.ones(shape=(t0, v,1)),
                          ), 0)
        
        mel_spectrogram = mel_spectrogram * mask
    return tf.cast(mel_spectrogram, dtype=tf.float32)


def random_brightness(image):
    return tf.image.random_brightness(image, 0.2)

def random_gamma(image):
    return tf.image.random_contrast(image, lower = 0.1, upper = 0.3)

def random_flip_right(image):
    return tf.image.random_flip_left_right(image)

def random_flip_up_down(image):
    return tf.image.random_flip_left_right(image)

available_ops = [
          frequency_masking ,
          time_masking, 
          random_brightness, 
          random_flip_up_down,
          random_flip_right 
         ]

def apply_augmentation(image, target):
    num_layers = int(np.random.uniform(low = 0, high = 3))
    
    for layer_num in range(num_layers):
        op_to_select = tf.random.uniform([], maxval=len(available_ops), dtype=tf.int32, seed = seed)
        for (i, op_name) in enumerate(available_ops):
            image = tf.cond(
            tf.equal(i, op_to_select),
            lambda selected_func=op_name,: selected_func(
                image),
            lambda: image)
    return image, target
                                                                                       

Audio Data Augmentation - https://www.tensorflow.org/io/tutorials/audio

In [ ]:
def preprocess(image, target):
    
    image = tf.image.grayscale_to_rgb(image)
    image = tf.image.resize(image, [height,width])
    image = tf.image.per_image_standardization(image)
    return image , target


In [ ]:
def load_dataset(filenames, labeled = True, ordered = False , training = True):
    # Read from TFRecords. For optimal performance, reading from multiple files at once and
    # Disregarding data order. Order does not matter since we will be shuffling the data anyway
    
    ignore_order = tf.data.Options()
    if not ordered:
        # disable order, increase speed
        ignore_order.experimental_deterministic = False 
        
    # automatically interleaves reads from multiple files
    dataset = tf.data.TFRecordDataset(filenames, num_parallel_reads = AUTO )
    # use data as soon as it streams in, rather than in its original order
    dataset = dataset.map(ReadLabeledAudio , num_parallel_calls = AUTO )
    dataset = dataset.map(transform_log_mel_spect , num_parallel_calls = AUTO)   
    if training:
        dataset = dataset.map(apply_augmentation, num_parallel_calls = AUTO)
    dataset = dataset.map(preprocess, num_parallel_calls = AUTO)
    return dataset

In [ ]:
def get_dataset(filenames, training = True):
    if training:
        dataset = load_dataset(filenames , training = True)
        dataset = dataset.shuffle(256).repeat()
        dataset = dataset.batch(GLOBAL_BATCH_SIZE, drop_remainder = True)
    else:
        dataset = load_dataset(filenames , training = False)
        dataset = dataset.batch(GLOBAL_BATCH_SIZE).cache()
    
    dataset = dataset.prefetch(AUTO)
    return dataset

In [ ]:
FILENAMES

In [ ]:
# mel spectrogram visualization
train_dataset = get_dataset(FILENAMES, training = True)

In [ ]:
train_dataset

In [ ]:
plt.figure(figsize=(16,6))
for i, (wav, target) in enumerate(train_dataset.unbatch().take(4)):
    plt.subplot(2,2,i+1)
    plt.imshow(wav[:, :, 0])
plt.show()

# 3. Model Building

### LWLRAP 
Label-weighted label-ranking average precision (lwlrap, pronounced "Lol wrap"). This measures the average precision of retrieving a ranked list of relevant labels for each test clip.<br/>
We use label weighting because it allows per-class values to be calculated, and still have the overall metric be expressed as simple average of the per-class metrics (weighted by each label's prior in the test set).
for more info - https://www.kaggle.com/pkmahan/understanding-lwlrap

In [ ]:
def _one_sample_positive_class_precisions(example):
    y_true, y_pred = example
    y_true = tf.reshape(y_true, tf.shape(y_pred))
    retrieved_classes = tf.argsort(y_pred, direction='DESCENDING')
#     shape = tf.shape(retrieved_classes)
    class_rankings = tf.argsort(retrieved_classes)
    retrieved_class_true = tf.gather(y_true, retrieved_classes)
    retrieved_cumulative_hits = tf.math.cumsum(tf.cast(retrieved_class_true, tf.float32))

    idx = tf.where(y_true)[:, 0]
    i = tf.boolean_mask(class_rankings, y_true)
    r = tf.gather(retrieved_cumulative_hits, i)
    c = 1 + tf.cast(i, tf.float32)
    precisions = r / c

    dense = tf.scatter_nd(idx[:, None], precisions, [y_pred.shape[0]])
    return dense

# @tf.function
class LWLRAP(tf.keras.metrics.Metric):
    def __init__(self, num_classes, name='lwlrap'):
        super().__init__(name=name)

        self._precisions = self.add_weight(
            name='per_class_cumulative_precision',
            shape=[num_classes],
            initializer='zeros',
        )

        self._counts = self.add_weight(
            name='per_class_cumulative_count',
            shape=[num_classes],
            initializer='zeros',
        )

    def update_state(self, y_true, y_pred, sample_weight=None):
        precisions = tf.map_fn(
            fn=_one_sample_positive_class_precisions,
            elems=(y_true, y_pred),
            dtype=(tf.float32),
        )

        increments = tf.cast(precisions > 0, tf.float32)
        total_increments = tf.reduce_sum(increments, axis=0)
        total_precisions = tf.reduce_sum(precisions, axis=0)

        self._precisions.assign_add(total_precisions)
        self._counts.assign_add(total_increments)        

    def result(self):
        per_class_lwlrap = self._precisions / tf.maximum(self._counts, 1.0)
        per_class_weight = self._counts / tf.reduce_sum(self._counts)
        overall_lwlrap = tf.reduce_sum(per_class_lwlrap * per_class_weight)
        return overall_lwlrap

    def reset_states(self):
        self._precisions.assign(self._precisions * 0)
        self._counts.assign(self._counts * 0)

In [ ]:
def cosine_decay_with_warmup(global_step,
                             learning_rate_base,
                             total_steps,
                             warmup_learning_rate=0.0,
                             warmup_steps= 0,
                             hold_base_rate_steps=0):
 
    if total_steps < warmup_steps:
        raise ValueError('total_steps must be larger or equal to '
                     'warmup_steps.')
    learning_rate = 0.5 * learning_rate_base * (1 + tf.cos(
        np.pi *
        (tf.cast(global_step, tf.float32) - warmup_steps - hold_base_rate_steps
        ) / float(total_steps - warmup_steps - hold_base_rate_steps)))
    if hold_base_rate_steps > 0:
        learning_rate = tf.where(
          global_step > warmup_steps + hold_base_rate_steps,
          learning_rate, learning_rate_base)
    if warmup_steps > 0:
        if learning_rate_base < warmup_learning_rate:
            raise ValueError('learning_rate_base must be larger or equal to '
                         'warmup_learning_rate.')
        slope = (learning_rate_base - warmup_learning_rate) / warmup_steps
        warmup_rate = slope * tf.cast(global_step,
                                    tf.float32) + warmup_learning_rate
        learning_rate = tf.where(global_step < warmup_steps, warmup_rate,
                               learning_rate)
    return tf.where(global_step > total_steps, 0.0, learning_rate,
                    name='learning_rate')


#dummy example
rng = [i for i in range(int(EPOCHS * STEPS_PER_EPOCH))]
WARMUP_STEPS =  int(WARMUP_EPOCHS * STEPS_PER_EPOCH)
y = [cosine_decay_with_warmup(x , LEARNING_RATE, len(rng), 1e-5, WARMUP_STEPS) for x in rng]

sns.set(style='whitegrid')
fig, ax = plt.subplots(figsize=(20, 6))
plt.plot(rng, y)

In [ ]:
# below code copied from - https://www.kaggle.com/ashusma/training-rfcx-tensorflow-tpu-effnet-b2#This-notebook-shows-the-training-of-RFCX-data-on-Tensorflow-TPU
# to apply learning rate schedule stepwise we need to subclass keras callback
# if we would have applied lr schedule epoch wise then it is not needed we can only call class learningrateschedule 

class WarmUpCosineDecayScheduler(tf.keras.callbacks.Callback):

    def __init__(self,
                 learning_rate_base,
                 total_steps,
                 global_step_init=0,
                 warmup_learning_rate=0.0,
                 warmup_steps=0,
                 hold_base_rate_steps=0,
                 verbose=0):

        super(WarmUpCosineDecayScheduler, self).__init__()
        self.learning_rate_base = learning_rate_base
        self.total_steps = total_steps
        self.global_step = global_step_init
        self.warmup_learning_rate = warmup_learning_rate
        self.warmup_steps = warmup_steps
        self.hold_base_rate_steps = hold_base_rate_steps
        self.verbose = verbose
        self.learning_rates = []

    def on_batch_end(self, batch, logs=None):
        self.global_step = self.global_step + 1
        lr = K.get_value(self.model.optimizer.lr)
        self.learning_rates.append(lr)

    def on_batch_begin(self, batch, logs=None):
        lr = cosine_decay_with_warmup(global_step=self.global_step,
                                      learning_rate_base=self.learning_rate_base,
                                      total_steps=self.total_steps,
                                      warmup_learning_rate=self.warmup_learning_rate,
                                      warmup_steps=self.warmup_steps,
                                      hold_base_rate_steps=self.hold_base_rate_steps)
        K.set_value(self.model.optimizer.lr, lr)
        if self.verbose > 0:
            print('\nBatch %05d: setting learning '
                  'rate to %s.' % (self.global_step + 1, lr.numpy()))
            

total_steps = int(EPOCHS * STEPS_PER_EPOCH)
# Compute the number of warmup batches or steps.
warmup_steps = int(WARMUP_EPOCHS * STEPS_PER_EPOCH)
warmup_learning_rate = WARMUP_LEARNING_RATE

In [ ]:
def Build_MODEL():
    waveform = Input(shape=(None,None,3), dtype=tf.float32)
    noisy_waveform = GaussianNoise(0.2)(waveform)
    model = efn.EfficientNetB2(include_top=False, weights='imagenet',) 
    model_output = model(noisy_waveform)
    model_output = GlobalAveragePooling2D()(model_output)
    dense = Dropout(dropout)(model_output)
    predictions = Dense(num_classes, activation = classifier_activation )(dense)
    model = Model(name='Efficientnet', inputs=waveform,outputs=[predictions])
    return model

In [ ]:
def get_model():
    with strategy.scope():
        model = Build_MODEL()
        model.summary()
        model.compile(optimizer = 'adam',
                                loss = tf.keras.losses.BinaryCrossentropy(label_smoothing = 0.1),
                                metrics = [LWLRAP(num_classes = num_classes),
                                ])
    return model

In [ ]:
skf = KFold(n_splits=10, shuffle=True, random_state=seed)
oof_pred = []; oof_labels = []; history_list = []

for fold,(idxT, idxV) in enumerate(skf.split(np.arange(10))):
    if tpu: tf.tpu.experimental.initialize_tpu_system(tpu)
    print(f'\nFOLD: {fold+1}')
    print(f'TRAIN: {idxT} VALID: {idxV}')

    # Create train and validation sets
    TRAIN_FILENAMES = [FILENAMES[x] for x in idxT]
    VALID_FILENAMES = [FILENAMES[x] for x in idxV]
    np.random.shuffle(TRAIN_FILENAMES)
    
    train_dataset =  get_dataset(TRAIN_FILENAMES, training=True,)
    validation_data= get_dataset(VALID_FILENAMES, training=False) 

    model = get_model()

    model_path = f'model_fold {fold}.h5'
    early_stopping = EarlyStopping(monitor = 'val_lwlrap', mode = 'max', 
                       patience = PATIENCE, restore_best_weights=True, verbose=1)

    # Create the Learning rate scheduler.
    cosine_warm_up_lr = WarmUpCosineDecayScheduler(learning_rate_base= LEARNING_RATE,
                                    total_steps= total_steps,
                                    warmup_learning_rate= warmup_learning_rate,
                                    warmup_steps= warmup_steps,
                                    hold_base_rate_steps=0)

    ## TRAIN
    history = model.fit(train_dataset, 
                        steps_per_epoch=STEPS_PER_EPOCH, 
                        callbacks=[early_stopping, cosine_warm_up_lr], 
                        epochs=EPOCHS,  
                        validation_data = validation_data,
                        verbose = 2).history

    history_list.append(history)
    # Save last model weights
    model.save_weights(model_path)

# OOF predictions
    ds_valid = get_dataset(VALID_FILENAMES, training = False)
    oof_labels.append([target.numpy() for frame, target in iter(ds_valid.unbatch())])
    x_oof = ds_valid.map(lambda frames, target: frames)
    oof_pred.append(np.argmax(model.predict(x_oof), axis=-1))

    ## RESULTS
    print(f"#### FOLD {fold+1} OOF Accuracy = {np.max(history['val_lwlrap']):.3f}")

# 4.Generating Test output & submitting them

In [ ]:
def get_test_dataset(filenames, training = False):
    
    dataset = tf.data.TFRecordDataset(filenames, num_parallel_reads = AUTO )  
    dataset = dataset.map(ReadUnlabeledAudio , num_parallel_calls = AUTO ).unbatch()
    dataset = dataset.map(lambda spec : transform_log_mel_spect(spec['audio_wav'], spec['recording_id']) , num_parallel_calls = AUTO)
    dataset = dataset.map(preprocess, num_parallel_calls = AUTO)
    return dataset.batch(GLOBAL_BATCH_SIZE*4).cache()

In [ ]:
test_predict = []

test_data = get_test_dataset(TEST_FILES, training = False)
test_audio = test_data.map(lambda frames, recording_id: frames)

for fold in range(N_FOLDS):
    model.load_weights(f'model_fold {fold}.h5')
    test_predict.append(model.predict(test_audio, verbose = 1 ))

In [ ]:
np.array(test_predict).shape

In [ ]:
SUB = pd.read_csv('../input/rfcx-species-audio-detection/sample_submission.csv')

predict = np.array(test_predict).reshape(N_FOLDS, len(SUB), 60 // TIME, num_classes)
predict = np.mean(np.max(predict ,axis = 2) , axis = 0)
# predict = np.mean(predict, axis =  0)

recording_id = test_data.map(lambda frames, recording_id: recording_id).unbatch()
# # all in one batch
test_ids = next(iter(recording_id.batch(len(SUB) * 60 // TIME))).numpy().astype('U').reshape(len(SUB), 60 // TIME)

pred_df = pd.DataFrame({ 'recording_id' : test_ids[:, 0],
             **{f's{i}' : predict[:, i] for i in range(num_classes)} })

In [ ]:
pred_df.sort_values('recording_id', inplace = True) 
pred_df.to_csv('submission.csv', index = False)  